In [1]:
import numpy as np

# Define Perceptron
- ReLU based perceptron
- has weight array, one output, and size of weight array
- calc() : calculate the output by using current input
- upd_weight() : update the weight

In [ ]:
class Perceptron:
    weight = []
    output = 0
    size = 0

    def __init__(self, weight, output, size):
        self.weight = weight
        self.output = output
        self.size = size

        assert len(self.weight) == self.size, "Perceptron weight size is not matched"

    def calc(self, input):
        assert len(input) == self.size, "Input size in perceptron is not matched"
        self.output = 0
        for i in range(0, self.size):
            self.output += max(input[i] * self.weight[i], 0)  # y=max(0,n)

    def upd_weight(self, weight):
        self.weight = weight

    def soft_calc(self, input):
        assert len(input) == self.size, "Input size in perceptron is not matched"
        self.output = 0
        for i in range(0, self.size):
            self.output += input[i] * self.weight[i]
        self.output = np.exp(self.output)  # y=exp(0x)

# Define Layer
- has node array(perceptrons), weight matrix, and input array(output of previous layer)
- set_input() : set input array
- init_weight() : initiallize the weight (-0.001 ~ 0.001)

In [ ]:
class Layer:
    nodes = []
    input = []
    w_mat = []
    output = []
    n_nodes = 0
    n_input = 0
    n_output = 0

    def __init__(self, n_nodes, n_input):
        self.n_nodes = n_nodes
        for i in range(0, self.n_nodes):
            self.nodes.append(Perceptron())
        assert len(self.nodes) == self.n_nodes, "# of nodes in Layer is not matched"

        self.n_input = n_input

    def set_input(self, input):
        self.input = input
        assert len(self.input) == self.n_input, "# of input in Layer is not matched"

    def upd_weight(self, w_mat):
        self.w_mat = w_mat
        for index in range(0, self.n_nodes):
            self.nodes[index].upd_weight(w_mat[:, index])

    def init_weight(self):
        new_mat = (np.random.rand(self.n_input, self.n_nodes) * 0.002) - 0.001
        self.upd_weight(new_mat)

    def calc_layer(self):
        assert len(self.input) > 0, "Input is empty in current layer"
        self.output = [node.calc(self.input) for node in self.nodes]
        self.n_output = len(self.output)

        assert len(self.output) == self.n_output, "# of output in Layer is not matched"

    def soft_layer(self):
        assert len(self.input) > 0, "Input is empty in current layer"
        self.output = [node.soft_calc(self.input) for node in self.nodes]
        sum_output = np.sum(self.output)
        self.output = [node / sum_output for node in self.output]
        self.n_output = len(self.output)

        assert len(self.output) == self.n_output, "# of output in Layer is not matched"


# Define Network

In [7]:
class Network:
    n_hidden = 0
    hidden = []  # 밑에서 부터 위로
    node_in_hidden = []
    n_input = 0
    input = []
    n_output = 0
    output = []
    result = []
    learning_rate = 0
    total_loss=[]
    error=0

    def __init__(self, n_hidden, n_input, n_output): # 2, 784, 10
        self.n_hidden = n_hidden
        self.n_input = n_input
        self.n_output = n_output

    def config_layer(self):
        self.node_in_hidden = []
        for index in range(0, self.n_hidden):
            temp_h = int(input(index + "번째 Layer node 갯수 : "))
            assert temp_h >= 0, "node is not positive"
            self.node_in_hidden.append(temp_h)
        nodes = np.array(self.n_input, self.node_in_hidden)
        for index in range(1, self.n_hidden):
            self.hidden.append(Layer(nodes[index], nodes[index - 1]))

        self.output = Layer(self.n_output, nodes[self.n_hidden])
        assert len(self.node_in_hidden) == self.n_hidden, "# of node in layer is not matched"

    def feed_forward(self, input):
        self.input = input
        # Layer 0 calc
        self.hidden[0].set_input(input)
        self.hidden[0].calc_layer()
        # hidden Layer calc
        for layer_index in range(1, self.n_hidden):
            self.hidden[layer_index].set_input(self.hidden[layer_index - 1].output)
            self.hidden[layer_index].calc_layer()
        # calc output
        self.output.set_input(self.hidden[self.n_hidden - 1].output)
        result = self.output.soft_layer()

    def calc_dcost(self,target):
        cost = [-1 * target[index] * (1 - self.result[index]) for index in range(0, self.n_output)]
        self.total_loss=self.total_loss+cost
        
    def calc_error(self,target):
        cost = [-1 * target[index] * np.log(self.result[index]) for index in range(0, self.n_output)]
        self.error=self.error+(np.sum(cost)/self.n_output)
        
    def back_propagation(self):
        # update softmax
        cur_mat = np.matmul(self.total_loss, self.output.w_mat)
        upd_mat = self.output.w_mat - self.learning_rate * np.matmul(np.transpose(self.output.input), self.total_loss)
        self.output.upd_weight(upd_mat)
        # update RELU
        for h_index in range(self.n_hidden - 1, -1):
            cur_layer = self.hidden[h_index]
            temp_mat = cur_layer.w_mat  # for next layer
            relu_result = [1 if result > 0 else 0 for result in cur_layer.output]
            if h_index == 0:
                upd_mat = cur_layer.w_mat - self.learning_rate * np.matmul(np.transpose(self.input), cur_mat)
            else:
                upd_mat = cur_layer.w_mat - self.learning_rate * np.matmul(
                    np.transpose(self.hidden[h_index - 1].output), cur_mat)
            for cur_node in range(0, self.node_in_hidden[h_index]):
                upd_mat[:, cur_node] = upd_mat[:, cur_node] * relu_result[cur_node]
            cur_mat = np.matmul(cur_mat, temp_mat)
            self.hidden[h_index].upd_weight(upd_mat)
    
    def training(self,data,target,epoch,alpha):
        #batch training
        self.learning_rate=alpha
        for i in range(0,epoch):
            self.error=0
            self.total_loss=np.zeros(self.n_output)
            for data_index in range(0,55001):
                self.feed_forward(data[data_index,:])
                self.calc_dcost(target[data_index,:])
                self.calc_error(target[data_index,:])
            self.total_loss=self.total_loss/55000
            self.error=self.error/55000
            print("Error {}".format(self.error))
            self.back_propagation()

In [1]:
"""Functions for downloading and reading MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets


ImportError: No module named 'tensorflow'